# CS 544 MP 1
This Notebook serves as a simple baseline for MP1

### Prerequisites
This document assumes that you can run jupyter notebook with python 3, with packages:
1. Numpy
2. Matplotlib

Installed


## Problem Statement
I have been asked when a Quasi-Newton method implemented using conjugate gradient is better than Polak Ribiere, or vice versa. I should very much like to know a better answer than the one I gave (that I always use Quasi-Newton first). Using some problems from your domain, and in groups no larger than four, produce an experimental answer to this question. You should compare small problems (10's-100's of variables) and large problems (1000's or more variables). You should compare cases where you can expect large third derivatives to cases where the Hessian shouldn't change much. Does the restart strategy in Polak-Ribiere make a difference?

Note that I am encouraging clever experimentation here, rather than blank coding efficiency. You should be able to get good implementations of each method from various locations, and it's fine to use those. Appropriate comparisons are the number of steps, the speed of convergence, the resources required, etc.

## Matches.json File Format 
### Top level
Top level consists of simple array of camera objects

See camera objects section for further details.
```
[
  Camera Object // see Camera Object
]
```


### Camera Object
Camera object contains multiple components.

R: Rotation matrix in extrinsic matrix
t: translation vector in extrinsic matrix
E: 3x4 extrinsic matrix
K: 3x3 intrinsic matrix

`K.dot(E)` would yield projection matrix (3x4)

Camera object also contains feature hash. Each feature in camera corresponds to unique local feature id that is used as reference.
Please see Feature Object for more details.

```
{
  R: Rotation Matrix, (3x3 list),
  t: translation, (3x1 list),
  E: extrinsic Matrix (3x4 list),
  K: intrinsic Matrix (3x3 list),
  W: image width
  H: image height
  Features: {
      LocalFeatureId: Feature Object // See Feature Object
  }
}
```
### Feature Object
Feature object contains 3 elements.

x: coordinate of feature in normalized image coordinate
y: coordinate of feature in normalized image coordinate
normalized image coordinage simply means that it is center-subtracted.

For instance, if x = 0, y = 100 in image of size W=1000, H=600,
the feature coordinage would be at 500 ( because 1000 / 2 + 0), 400 ( because 600 / 2 + 100)

Feature object also contains Match objects. See Match Objects for details



```
{
  x: x coordinate of feature away from image center (principal offset),
  y: y coordinate of feature away from image center
  Matches: [
    Match Object
  ]
}
```

### Match Object
Match Object contains 2 things
SrcCameraIndex: index of camera that is matched with
SrcCameraFeatureIndex: index of feature in source camera's local feature index.
(If you don't know what local feature index is, please refer to Camera Object section LocalFeatureId)


```
{
   SrcCameraIndex: index of camera
   SrcCameraFeatureIndex: index of features in SourceCamera.features   
}
```

## Examples
### File IO
#### Reading JSON
```
import json
with open('matches.json', 'r') as f:
    camera_json = json.load(f)
```
#### Camera Object parsing

```
i = 0
E = np.array(camera_json[int(i)]['E'])
K = np.array(camera_json[int(i)]['K'])
# projection matrix
P = K.dot(E)
```
### Visualization
#### Camera mesh creation
```
def create_camera_mesh(camera_object):
    """
    Takes camera object as input, returns 3x12 camera mesh matrix
    """
    R = np.array(camera_object['R'])
    t = np.array(camera_object['t'])
    CC = -R.dot(t)
    o = [0, 0, 0]
    lu = [-1, 1, 1]
    ru = [1, 1, 1]
    rd = [1, -1, 1]
    ld = [-1, -1, 1]
    corners = np.array([
        o, lu, ru, o,
        ru, rd, o,
        rd, ld, o,
        ld, lu
    ]).T
    CCs = corners + t.reshape(3, 1)
    camera_mesh = -R.dot(CCs)
    return camera_mesh
```
#### Visualizing mesh
```
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.axis('equal')
for camera_id, camera_object in camera_json.items():
    camera_mesh = create_camera_mesh(camera_object)
    ax.plot(camera_mesh[0], camera_mesh[1], zs=camera_mesh[2])
```

#### Visualizing Points
```
test_points = np.load('points.npy')
ax.scatter(test_points[0], test_points[1], test_points[2])
```